In [1]:
!python -m pip install --upgrade pip --quiet
!pip install -U awscli --quiet

In [2]:
from sagemaker import get_execution_role
import sagemaker
import boto3
import os

boto_session = boto3.session.Session()
sm_session = sagemaker.session.Session()
region = boto_session.region_name
sm_client = boto_session.client("sagemaker")
role = get_execution_role()
s3_bucket = sm_session.default_bucket()

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/nargokul/Library/Application Support/sagemaker/config.yaml


In [3]:
from src.generated.shapes import ContainerDefinition, ProductionVariant, AsyncInferenceConfig, AsyncInferenceOutputConfig, AsyncInferenceClientConfig
# Endpoint Invoking 
from src.generated.resources import Model, EndpointConfig, Endpoint, TrainingJob
from time import gmtime, strftime

key = f'xgboost-iris-{strftime("%H-%M-%S", gmtime())}'
print("key", key)
os.environ['SAGEMAKER_ADMIN_CONFIG_OVERRIDE'] = '/Users/nargokul/workspace/sagemaker-code-gen/sample/sagemaker/2017-07-24/default-configs-inference.json'

/Users/nargokul/workspace/sagemaker-code-gen/venv/lib/python3.9/site-packages/pydantic/_internal/_fields.py:160: UserWarning: Field "model_access_config" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/Users/nargokul/workspace/sagemaker-code-gen/venv/lib/python3.9/site-packages/pydantic/_internal/_fields.py:160: UserWarning: Field "model_data_url" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/Users/nargokul/workspace/sagemaker-code-gen/venv/lib/python3.9/site-packages/pydantic/_internal/_fields.py:160: UserWarning: Field "model_data_source" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/Users/nargokul/workspace/sagemaker-code-gen/venv/lib/pytho

key xgboost-iris-02-01-22


/Users/nargokul/workspace/sagemaker-code-gen/venv/lib/python3.9/site-packages/pydantic/_internal/_fields.py:160: UserWarning: Field "model_bias_app_specification" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/Users/nargokul/workspace/sagemaker-code-gen/venv/lib/python3.9/site-packages/pydantic/_internal/_fields.py:160: UserWarning: Field "model_bias_job_input" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/Users/nargokul/workspace/sagemaker-code-gen/venv/lib/python3.9/site-packages/pydantic/_internal/_fields.py:160: UserWarning: Field "model_bias_job_output_config" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/Users/nargokul/workspace/sagemak

In [4]:
bucket_prefix = "async-inference-demo"
resource_name = "AsyncInferenceDemo-{}-{}"

model_s3_key = f"{bucket_prefix}/demo-xgboost-model.tar.gz"
model_url = f"s3://{s3_bucket}/{model_s3_key}"
print(f"Uploading Model to {model_url}")

with open("model/demo-xgboost-model.tar.gz", "rb") as model_file:
    boto_session.resource("s3").Bucket(s3_bucket).Object(model_s3_key).upload_fileobj(model_file)


from sagemaker import image_uris

# Specify an AWS container image and region as desired
container = image_uris.retrieve(region=region, framework="xgboost", version="0.90-1")

model = Model.create(
    model_name=key,
    primary_container=ContainerDefinition(
        image=container,
        model_data_url=f"s3://{s3_bucket}/{model_s3_key}"
    ),
    execution_role_arn=role,
)

Uploading Model to s3://sagemaker-us-west-2-211125564141/async-inference-demo/demo-xgboost-model.tar.gz


INFO:sagemaker.image_uris:Defaulting to only available Python version: py3
INFO:sagemaker.image_uris:Defaulting to only supported image scope: cpu.
INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials


Fetched defaults config from location: %s /Users/nargokul/workspace/sagemaker-code-gen/sample/sagemaker/2017-07-24/default-configs-inference.json
Not applying SDK defaults from location: %s /Users/nargokul/Library/Application Support/sagemaker/config.yaml
Fetched defaults config from location: %s /Users/nargokul/workspace/sagemaker-code-gen/sample/sagemaker/2017-07-24/default-configs-inference.json
Not applying SDK defaults from location: %s /Users/nargokul/Library/Application Support/sagemaker/config.yaml
{'CreationTime': datetime.datetime(2024, 5, 28, 19, 1, 23, 759000, tzinfo=tzlocal()),
 'EnableNetworkIsolation': False,
 'ExecutionRoleArn': 'arn:aws:iam::211125564141:role/Admin',
 'ModelArn': 'arn:aws:sagemaker:us-west-2:211125564141:model/xgboost-iris-02-01-22',
 'ModelName': 'xgboost-iris-02-01-22',
 'PrimaryContainer': {'Image': '246618743249.dkr.ecr.us-west-2.amazonaws.com/sagemaker-xgboost:0.90-1-cpu-py3',
                      'Mode': 'SingleModel',
                      'Mod

In [5]:
from datetime import datetime


endpoint_config = EndpointConfig.create(
    endpoint_config_name=key,
    production_variants=[
        ProductionVariant(
            variant_name="variant1",
            model_name=key,
            instance_type='ml.m5.xlarge',
            initial_instance_count=1
        )
    ],
    async_inference_config=AsyncInferenceConfig(
        output_config=AsyncInferenceOutputConfig(s3_output_path=f"s3://{s3_bucket}/{bucket_prefix}/output"),
        client_config=AsyncInferenceClientConfig(
            max_concurrent_invocations_per_instance=4
        )
    )
)


Fetched defaults config from location: %s /Users/nargokul/workspace/sagemaker-code-gen/sample/sagemaker/2017-07-24/default-configs-inference.json
Not applying SDK defaults from location: %s /Users/nargokul/Library/Application Support/sagemaker/config.yaml
{'AsyncInferenceConfig': {'ClientConfig': {'MaxConcurrentInvocationsPerInstance': 4},
                          'OutputConfig': {'S3OutputPath': 's3://sagemaker-us-west-2-211125564141/async-inference-demo/output'}},
 'CreationTime': datetime.datetime(2024, 5, 28, 19, 1, 24, 285000, tzinfo=tzlocal()),
 'EnableNetworkIsolation': False,
 'EndpointConfigArn': 'arn:aws:sagemaker:us-west-2:211125564141:endpoint-config/xgboost-iris-02-01-22',
 'EndpointConfigName': 'xgboost-iris-02-01-22',
 'ProductionVariants': [{'InitialInstanceCount': 1,
                         'InitialVariantWeight': 1.0,
                         'InstanceType': 'ml.m5.xlarge',
                         'ModelName': 'xgboost-iris-02-01-22',
                         'Vari

In [6]:
endpoint = Endpoint.create(endpoint_name=key, endpoint_config_name=key)

Fetched defaults config from location: %s /Users/nargokul/workspace/sagemaker-code-gen/sample/sagemaker/2017-07-24/default-configs-inference.json
Not applying SDK defaults from location: %s /Users/nargokul/Library/Application Support/sagemaker/config.yaml
{'AsyncInferenceConfig': {'ClientConfig': {'MaxConcurrentInvocationsPerInstance': 4},
                          'OutputConfig': {'S3OutputPath': 's3://sagemaker-us-west-2-211125564141/async-inference-demo/output'}},
 'CreationTime': datetime.datetime(2024, 5, 28, 19, 1, 24, 770000, tzinfo=tzlocal()),
 'EndpointArn': 'arn:aws:sagemaker:us-west-2:211125564141:endpoint/xgboost-iris-02-01-22',
 'EndpointConfigName': 'xgboost-iris-02-01-22',
 'EndpointName': 'xgboost-iris-02-01-22',
 'EndpointStatus': 'Creating',
 'LastModifiedTime': datetime.datetime(2024, 5, 28, 19, 1, 24, 770000, tzinfo=tzlocal()),
 'ResponseMetadata': {'HTTPHeaders': {'content-length': '449',
                                      'content-type': 'application/x-amz-json

In [7]:
def upload_file(input_location):
    prefix = f"{bucket_prefix}/input"
    return sm_session.upload_data(
        input_location,
        bucket=sm_session.default_bucket(),
        key_prefix=prefix,
        extra_args={"ContentType": "text/libsvm"},
    )

input_1_location = "input/test_point_0.libsvm"
input_1_s3_location = upload_file(input_1_location)



In [9]:
endpoint = Endpoint.get(endpoint_name=key)
response= endpoint.invoke_async(input_location=input_1_s3_location)

INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials


{'AsyncInferenceConfig': {'ClientConfig': {'MaxConcurrentInvocationsPerInstance': 4},
                          'OutputConfig': {'S3OutputPath': 's3://sagemaker-us-west-2-211125564141/async-inference-demo/output'}},
 'CreationTime': datetime.datetime(2024, 5, 28, 19, 1, 24, 770000, tzinfo=tzlocal()),
 'EndpointArn': 'arn:aws:sagemaker:us-west-2:211125564141:endpoint/xgboost-iris-02-01-22',
 'EndpointConfigName': 'xgboost-iris-02-01-22',
 'EndpointName': 'xgboost-iris-02-01-22',
 'EndpointStatus': 'InService',
 'LastModifiedTime': datetime.datetime(2024, 5, 28, 19, 3, 59, 129000, tzinfo=tzlocal()),
 'ProductionVariants': [{'CurrentInstanceCount': 1,
                         'CurrentWeight': 1.0,
                         'DeployedImages': [{'ResolutionTime': datetime.datetime(2024, 5, 28, 19, 1, 25, 314000, tzinfo=tzlocal()),
                                             'ResolvedImage': '246618743249.dkr.ecr.us-west-2.amazonaws.com/sagemaker-xgboost@sha256:4814427c3e0a6cf99e637704da3ada0